In [83]:
import tensorflow.compat.v1 as tf
from tensorflow import feature_column
from tensorflow.keras import layers
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

In [84]:
seed = 0
# tensorflow, numpy 랜덤 값을 설정합니다.
tf.set_random_seed(seed)
np.random.seed(seed)

In [85]:
#doginfo.csv파일 데이터를 pandas를 이용해 읽어옵니다.
dog_data=pd.read_csv("doginfo_mmy.csv")
dog_train=pd.read_csv("testDog.csv")
kindCd=pd.read_csv("kindCd.csv")

dog_data = dog_data.dropna(axis=0)
dog_train = dog_train.dropna(axis=0)
kindCd_data = kindCd.dropna(axis=0)

dog_data
# print(dog_train)
# print(kindCd)

,kindNum,neuterYn,sexCd,weight,noticeDays,age2,processState
0,128.0,N,F,7.46,10,12,0
1,114.0,N,M,7.00,14,1,1
2,114.0,U,M,4.50,11,2,0
3,67.0,N,M,10.00,8,1,0
4,114.0,N,M,6.00,8,4,0
...,...,...,...,...,...,...,...
1994,114.0,N,M,0.98,10,0,1
1995,114.0,N,F,1.20,10,0,1
1996,106.0,N,M,50.00,10,3,1
1997,114.0,U,F,9.00,10,3,0


In [86]:
kindCd = np.array(kindCd_data, dtype = np.float64)
kindCd_train=np.array(kindCd_data, dtype=np.float64)

kindCd = kindCd.reshape(177)
kindCd_train=kindCd_train.reshape(177)
print(kindCd)
print(kindCd.shape)

[ 54.  56.  55. 118. 115.  37.  81. 204.  83.  82.  38.  39.  40.  43.
  42. 153.  41. 120. 155.  69.  71. 142.  93. 167.  70. 166.  94. 121.
 152.  73. 146.  72. 159.  76.  75.  79.  78.  77.  74.  80. 114. 133.
  12.  17.  15. 164. 157. 148.  16.  20.  21.  22.  24. 208.  23.  26.
  27. 169.  25.  19.  13.  18.  14. 162.  85.  96.  95.   1.  34. 104.
  31.  99. 122. 123.  97. 132. 105. 154. 124. 100. 103. 151. 139. 101.
 102.  98. 136. 202. 160. 203.   8. 131.   9. 119. 150. 210.  57.  58.
  59.   6.   4.   7.   5. 143.  11.  10. 137.  84. 163. 112. 113. 149.
 211. 110. 205. 108. 109.  60.  46.  47.  44.  45.  53.  62.  61.  52.
 165.  51. 156. 129.  67.  35.  33.  32. 158. 144.  30.  29.  64. 207.
  28.   2.  68. 125. 141. 145.  36.  66.  65.  63. 140. 107. 106. 209.
  86.  88.  90.  87. 138.  89. 126. 127. 128.  91.   3. 161.  50. 168.
  49. 147.  92.  48. 135. 206. 130. 134. 111.]
(177,)


In [87]:
# kindNum을 원핫 인코딩
kindCd = pd.concat((pd.get_dummies(dog_data.kindNum, columns=kindCd), pd.DataFrame(columns=kindCd))).fillna(0)
# kindCd

# 학습데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
dog_data.drop(['kindNum'], axis='columns', inplace=True)
dog_data = pd.concat([dog_data, kindCd], axis=1)


# kindNum을 원핫 인코딩
kindCd_train = pd.concat((pd.get_dummies(dog_train.kindNum, columns=kindCd_train), pd.DataFrame(columns=kindCd_train))).fillna(0)

# 테스트데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
dog_train.drop(['kindNum'], axis='columns', inplace=True)
dog_train = pd.concat([dog_train, kindCd_train], axis=1)
dog_data

,neuterYn,sexCd,weight,noticeDays,age2,processState,1.0,2.0,3.0,4.0,...,202.0,203.0,204.0,205.0,206.0,207.0,208.0,209.0,210.0,211.0
0,N,F,7.46,10,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,N,M,7.00,14,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,U,M,4.50,11,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,N,M,10.00,8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,N,M,6.00,8,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,N,M,0.98,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1995,N,F,1.20,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1996,N,M,50.00,10,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1997,U,F,9.00,10,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis = 1)
        data = data.drop(column, axis = 1)
    return data

In [89]:
dummy_columns = ["neuterYn", "sexCd"]
data = dummy_data(dog_data, dummy_columns)
train_data = dummy_data(dog_train, dummy_columns)
data.head()
train_data.head()

print(data)

data = np.array(data, dtype = np.float64)
train_data = np.array(train_data, dtype = np.float64)


      weight  noticeDays  age2  processState  1.0  2.0  3.0  4.0  5.0  6.0  \
0       7.46          10    12             0    0    0    0    0    0    0   
1       7.00          14     1             1    0    0    0    0    0    0   
2       4.50          11     2             0    0    0    0    0    0    0   
3      10.00           8     1             0    0    0    0    0    0    0   
4       6.00           8     4             0    0    0    0    0    0    0   
...      ...         ...   ...           ...  ...  ...  ...  ...  ...  ...   
1994    0.98          10     0             1    0    0    0    0    0    0   
1995    1.20          10     0             1    0    0    0    0    0    0   
1996   50.00          10     3             1    0    0    0    0    0    0   
1997    9.00          10     3             0    0    0    0    0    0    0   
1998   12.50          10     3             0    0    0    0    0    0    0   

      ...  208.0  209.0  210.0  211.0  neuterYn_N  neuterYn_U  

In [90]:
a = data[:, :3]
b = data[:, 4:]

#numpy 배열에서 데이터 변화요인(kindCd, neuterYn, sexCd, weight, noticeDays, age2)으로 사용할 데이터를 뽑아냅니다.
xData = np.concatenate([a, b], axis = 1)

a= train_data[:, :3]
b= train_data[:, 4:]

testX=np.concatenate([a, b], axis=1)

# print(xData)
# type(xData)
# xData.shape
testX

array([[ 0.8, 10. ,  0. , ...,  0. ,  1. ,  0. ],
       [ 5. , 10. ,  1. , ...,  0. ,  1. ,  0. ],
       [ 5. , 10. , 13. , ...,  1. ,  0. ,  0. ],
       ...,
       [ 1.7, 11. ,  0. , ...,  1. ,  0. ,  0. ],
       [ 3.6, 11. ,  2. , ...,  1. ,  0. ,  0. ],
       [ 3.1, 11. ,  2. , ...,  0. ,  1. ,  0. ]])

In [91]:
#numpy배열에서 결과(입양여부)로 사용할 데이터를 뽑아냅니다.
yData=data[:,[3]]
testY=data[:,[3]]
print(yData)
type(yData)
yData.dtype

[[0.]
 [1.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


dtype('float64')

In [96]:
keep_drop=0.5
#뽑아낸 데이터를 tensorflow로 처리하기 위해 placeholder를 만듭니다.
X=tf.placeholder(tf.float64, [None, 186])
Y=tf.placeholder(tf.float64, [None, 1])

#다변인 선형 회귀 모델의 기울기와 y절편을 임의의 값으로 초기화합니다.
W1=tf.Variable(tf.random_normal([186, 1], dtype=tf.float64))#6행 1열 난수를 발생시킵니다.
b1=tf.Variable(tf.random_normal([1], dtype=tf.float64)) #1행 1열 난수를 발생시킵니다.
L1=tf.nn.relu(tf.matmul(X, W1)+b1)
L1=tf.nn.dropout(L1, keep_prob=keep_drop)

W2=tf.Variable(tf.random_normal([186, 1], dtype=tf.float64))#6행 1열 난수를 발생시킵니다.
b2=tf.Variable(tf.random_normal([1], dtype=tf.float64)) #1행 1열 난수를 발생시킵니다.
L2=tf.nn.relu(tf.matmul(X, W2)+b2)
L2=tf.nn.dropout(L2, keep_prob=keep_drop)

W3=tf.Variable(tf.random_normal([186, 1], dtype=tf.float64))#6행 1열 난수를 발생시킵니다.
b3=tf.Variable(tf.random_normal([1], dtype=tf.float64)) #1행 1열 난수를 발생시킵니다.
y=tf.nn.relu(tf.matmul(X, W3)+b3)


In [97]:
# 시그모이드 방정식을 만듭니다.
# y = tf.sigmoid(tf.matmul(X, a) + b)
# 오차 함수
cost = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y))
optimizer=tf.train.GradientDescentOptimizer(0.01).minimize(cost)
# optimizer=tf.contrib.estimator.clip_gradients_by_norm(optimizer, 5.0)
predicted = tf.cast(y >= 0.5, dtype = tf.float64)

# 예상값이 실제값과 일치하는 정도(정확도)를 계산합니다.
# tf.equal(): tensorflow에서 인수로 지정된 값이 같으면 True, 다르면 False 결과를 tf.float64로 캐스팅 시키고 시행된 전체 평균을 계산합니다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float64))  # 정확도 (0 또는 1) 

# with tf.Session() as sess:  # 세션 구간을 만듦
#     sess.run(tf.global_variables_initializer())
#     for i in range(10001):
#         # count : 학습 횟수, a1 : 기울기(공부 시간), asess.run(cost, feed_dict={X:xData, Y:yData}))
#         a_, b_, cost_, _ = sess.run([a, b, cost, optimizer], feed_dict = {X:xData, Y:yData})
#         if i % 500 == 0:
#             print(sess.run(y, feed_dict={X:xData}))
#             print('count:{}, b = {}, loss = {}'.format(i, b_, cost_)) # 학습시킨 데이터 출력
#     av=sess.run([accuracy], feed_dict={X:xData, Y:yData})
#     print("정확도: {}".format(av))
                
#     for i in range(len(train_data)):
#         new_x=train_data[i, :].reshape(1,186)
#         result, new_y = sess.run([predicted, y], feed_dict={X:new_x})
#         print('입양 여부 : %s,    입양 확률 : %8.4f%%' % ('입양' if result == 1 else '안락사', new_y*100))
    
#     saver = tf.train.Saver()
#     save_path = saver.save(sess, './dog.ckpt')
#     print('학습된 모델을 저장했습니다')
step_list=[]
cost_list=[]
y_list=[]
pred_list=[]
acc_list=[]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        cost_, _ = sess.run([cost, optimizer], feed_dict={X:xData, Y:yData})
        
        if step %200 == 0:
            print('step{:4} : cost={:6.4f}'.format(step, cost_))
    h, c, a = sess.run([y, predicted, accuracy], feed_dict={X:xData, Y:yData})
    print("\n y : {}, correct: {}, Accuracy: {}".format(h, c, a))
    
    saver = tf.train.Saver()
    save_path = saver.save(sess, './dogtensor2.ckpt')
    print('학습된 모델을 저장했습니다')

step   0 : cost=   nan
step 200 : cost=   nan
step 400 : cost=   nan
step 600 : cost=   nan
step 800 : cost=   nan
step1000 : cost=   nan
step1200 : cost=   nan
step1400 : cost=   nan
step1600 : cost=   nan
step1800 : cost=   nan
step2000 : cost=   nan
step2200 : cost=   nan
step2400 : cost=   nan
step2600 : cost=   nan
step2800 : cost=   nan
step3000 : cost=   nan
step3200 : cost=   nan
step3400 : cost=   nan
step3600 : cost=   nan
step3800 : cost=   nan
step4000 : cost=   nan
step4200 : cost=   nan
step4400 : cost=   nan
step4600 : cost=   nan
step4800 : cost=   nan
step5000 : cost=   nan
step5200 : cost=   nan
step5400 : cost=   nan
step5600 : cost=   nan
step5800 : cost=   nan
step6000 : cost=   nan
step6200 : cost=   nan
step6400 : cost=   nan
step6600 : cost=   nan
step6800 : cost=   nan
step7000 : cost=   nan
step7200 : cost=   nan
step7400 : cost=   nan
step7600 : cost=   nan
step7800 : cost=   nan
step8000 : cost=   nan
step8200 : cost=   nan
step8400 : cost=   nan
step8600 : 

In [ ]:
#뽑아낸 데이터를 tensorflow로 처리하기 위해 placeholder를 만듭니다.
X2=tf.placeholder(tf.float64, [None, 186])
Y2=tf.placeholder(tf.float64, [None, 1])

#다변인 선형 회귀 모델의 기울기와 y절편을 임의의 값으로 초기화합니다.
# a=tf.Variable(tf.random_normal([186, 1], dtype=tf.float64))#6행 1열 난수를 발생시킵니다.
# b=tf.Variable(tf.random_normal([1], dtype=tf.float64)) #1행 1열 난수를 발생시킵니다.
# print(X)
# print(Y)

In [41]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
    
#     for step in range(1000):
#         sess.run(optimizer, feed_dict={X:testX, Y:testY})
        
#         for step in range(10001):
#             cost_, _ = sess.run([cost, optimizer], feed_dict={X:testX, Y:testY})
        
#         if step %200 == 0:
#             print('step{:4} : cost={:6.4f}'.format(step, cost_))
#     h, c, a = sess.run([y, predicted, accuracy], feed_dict={X:testX, Y:testY})
#     print("\n y : {}, correct: {}, Accuracy: {}".format(h, c, a))
        

InvalidArgumentError: Incompatible shapes: [22777,1] vs. [610,1]
	 [[node gradients_4/mul_8_grad/BroadcastGradientArgs (defined at C:\Users\auddu\anaconda3\envs\auddus\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]

Original stack trace for 'gradients_4/mul_8_grad/BroadcastGradientArgs':
  File "C:\Users\auddu\anaconda3\envs\auddus\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\auddu\anaconda3\envs\auddus\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\auddu\anaconda3\envs\auddus\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\auddu\anaconda3\envs\auddus\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\auddu\anaconda3\envs\auddus\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
    self.io_loop.start()
  File "C:\Users\auddu\anaconda3\envs\auddus\lib\s